<a href="https://colab.research.google.com/github/Himanshu0702/Machine-Learning/blob/main/Tlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import numpy as np
import cv2
import PIL
import PIL.Image as Image
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [6]:
pip install split-folders[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#Split the data into train,test and validation
import splitfolders
#Location of data
input_folder = '/content/drive/MyDrive/Skin_Disease'
#Location of train,test and validation data is to be given in output
splitfolders.ratio(input_folder, output="/content/drive/MyDrive/dataset",
                   seed=42, ratio=(.7, .2, .1),
                   group_prefix = None)

Copying files: 3592 files [19:08,  3.13 files/s]


In [8]:
train = '/content/drive/MyDrive/dataset/train'
test = '/content/drive/MyDrive/dataset/test'
validation = '/content/drive/MyDrive/dataset/val'

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

    #splits data into training and testing(validation) sets
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/dataset/train',  # Source directory
        target_size=(224, 224),  # Resizes images
        batch_size=15,
        class_mode='sparse')

test_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/dataset/test',  # Source directory
        target_size=(224, 224),  # Resizes images
        batch_size=15,
        class_mode='sparse')
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/val',
    target_size=(224, 224),
    batch_size=15,
    class_mode='sparse')

Found 2513 images belonging to 4 classes.
Found 362 images belonging to 4 classes.
Found 717 images belonging to 4 classes.


In [10]:
feature_extractor = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [11]:
feature_extractor_layer = hub.KerasLayer(feature_extractor, input_shape=(224,224,3))

In [12]:
feature_extractor_layer.trainable = False

In [13]:
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [14]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])


In [15]:
history = model.fit(train_generator, epochs=6, validation_data = validation_generator)

Epoch 1/6


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


168/168 [==============================] - 134s 753ms/step - loss: 1.2290 - acc: 0.4775 - val_loss: 0.9335 - val_acc: 0.6165
Epoch 2/6
168/168 [==============================] - 119s 711ms/step - loss: 0.9760 - acc: 0.5758 - val_loss: 0.8775 - val_acc: 0.6318
Epoch 3/6
168/168 [==============================] - 118s 704ms/step - loss: 0.8938 - acc: 0.6152 - val_loss: 0.8656 - val_acc: 0.6416
Epoch 4/6
168/168 [==============================] - 121s 721ms/step - loss: 0.8312 - acc: 0.6558 - val_loss: 0.8424 - val_acc: 0.6444
Epoch 5/6
168/168 [==============================] - 120s 714ms/step - loss: 0.8097 - acc: 0.6510 - val_loss: 0.9092 - val_acc: 0.6123
Epoch 6/6
168/168 [==============================] - 119s 710ms/step - loss: 0.7593 - acc: 0.6741 - val_loss: 0.8265 - val_acc: 0.6597


In [16]:
result = model.evaluate(test_generator)

25/25 [==============================] - 13s 532ms/step - loss: 0.8437 - acc: 0.6713


In [17]:
img = cv2.imread('/content/drive/MyDrive/dataset/test/Eczema Photos/03EczemaExcoriated011204.jpg')
resize = tf.image.resize(img, (224,224))
yhat = model.predict(np.expand_dims(resize/255,0))
yhat

1/1 [==============================] - 1s 579ms/step


array([[0.08947213, 0.49577546, 0.08654491, 0.32820752]], dtype=float32)

In [18]:
train_generator.class_indices

{'Atopic Dermatitis Photos': 0,
 'Eczema Photos': 1,
 'Melanoma Skin Cancer Nevi and Moles': 2,
 'Psoriasis pictures Lichen Planus and related diseases': 3}